In [1]:
from utils import *
import pandas as pd

In [2]:
channels = pd.read_csv(level2_folder + 'channel.csv')
funding_txs = read_json(level1_folder + 'blockstream_funding_txs.json')
settlement_txs = read_json(level1_folder + 'blockstream_settlement_txs.json')
funded_address_settlement_txs = read_json(level1_folder + 'funded_address_blockstream_settlement_txs.json')
funding_address_entity = read_json(level1_folder + 'funding_address_entity.json')
settlement_address_entity = read_json(level1_folder + 'settlement_address_entity.json')
input_file1 = results_folder + 'filtered_all_heuristic_2_entity_node_conf.json'
heuristic_2_entity_node = read_json(input_file1, double_int_key=True)[2]
input_file3 = results_folder + 'filtered_all_heuristic_2_node_entity_conf.json'
heuristic_2_node_entity_conf = read_json(input_file3, True)[2]
input_file2 = results_folder + 'filtered_all_heuristic_2_entity_node_conf_old.json'
heuristic_2_entity_node_old = read_json(input_file2, double_int_key=True)[2]

In [5]:
open_channels = 0
for fa in funded_address_settlement_txs:
    if not funded_address_settlement_txs[fa]:
        open_channels += 1
print(open_channels, len(channels))

37996 98431


In [3]:
n_addresses = len(set(funding_address_entity.keys()).union(set(settlement_address_entity.keys())))
n_addresses

238070

In [6]:
len(set([e for e in settlement_address_entity.values()]).union(set([e for e in funding_address_entity.values()])))

138457

In [7]:
len(set(funding_address_entity.values()))

96181

In [8]:
len(set(settlement_address_entity.values()))

53370

In [9]:
###############
node_channels = dict()
for channel in channels.values:
    c, n1, n2 = channel
    if n1 not in node_channels:
        node_channels[n1] = set()
    node_channels[n1].add(c)
    if n2 not in node_channels:
        node_channels[n2] = set()
    node_channels[n2].add(c)

In [10]:
len(node_channels)

10910

In [13]:
funding_settlement_txs = set()
for h in funding_txs:
    funding_settlement_txs.add(h)
for h in settlement_txs:
    funding_settlement_txs.add(h)
len(funding_settlement_txs)

158687

In [14]:
len(settlement_txs)

60447

In [17]:
###############
# to see distribution of #outputs in settlement txs
n_outputs_stx = dict()
open_channels = 0
for fa in funded_address_settlement_txs:
    if not funded_address_settlement_txs[fa]:
        open_channels += 1
    else:
        for st in funded_address_settlement_txs[fa]:
            n_outputs = len(st['vout'])
            if n_outputs not in n_outputs_stx:
                n_outputs_stx[n_outputs] = []
            n_outputs_stx[n_outputs].append(st['txid'])

In [22]:
n_less2 = round(41.8 + 56.3, 2)
n_more2 = 100 - n_less2
print(n_less2, n_more2)

98.1 1.9000000000000057


In [19]:
closed_channels = len(settlement_txs)
# distribution of #outputs in settlement txs
for k in n_outputs_stx:
    print(k, len(n_outputs_stx[k]), 100*round(len(n_outputs_stx[k])/closed_channels, 3))

2 25285 41.8
1 34045 56.3
4 161 0.3
3 847 1.4000000000000001
6 22 0.0
5 47 0.1
11 3 0.0
9 7 0.0
7 6 0.0
31 1 0.0
10 1 0.0
22 1 0.0
8 7 0.0
16 1 0.0
14 1 0.0


{'txid': '8007158bf82842459bf18b0237bed2c021f51476a82fc942fdc3070118b4eb44',
 'version': 1,
 'locktime': 0,
 'vin': [{'txid': 'de4bea2d8f1560d4f548e94ad2df0ef97349c3b3a930f84e2ea7ffdfe60ac007',
   'vout': 0,
   'prevout': {'scriptpubkey': '001465ceb80d6770e1dc85598303f9e4b74ecf98b093',
    'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_20 65ceb80d6770e1dc85598303f9e4b74ecf98b093',
    'scriptpubkey_type': 'v0_p2wpkh',
    'scriptpubkey_address': 'bc1qvh8tsrt8wrsaep2esvplne9hfm8e3vynn9wd65',
    'value': 7016237},
   'scriptsig': '',
   'scriptsig_asm': '',
   'witness': ['3044022057e862eb95b1c09370322dc9bb64fd8a3337a69729fac9d277525585719f5d9e02207731baa919df60ec97273642093e51869c788ffeacfb4b58b3644a667e01d32301',
    '020fb656af535132244c949ff6ba76f76b0972941cf71e4c94fff837a8ffae3227'],
   'is_coinbase': False,
   'sequence': 4294967295}],
 'vout': [{'scriptpubkey': '00143d4438b8a625c2c52da6f6bc0953291f53f2d044',
   'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_20 3d4438b8a625c2c52da6f6bc0953291f53f

In [32]:
latest_funding_tx = 0
for tx in funding_txs.values():
    t = tx['status']['block_time']
    if t > latest_funding_tx:
        latest_funding_tx = t
ts2date(latest_funding_tx)

'2020-09-09 23:41:33'

In [31]:
import datetime
def ts2date(ts):
    return datetime.datetime.fromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')
def ts2object(ts):
    return datetime.datetime.fromtimestamp(int(ts))

In [ ]:
on_chain_heuristics = {och: (True if och != 'none' else False) for och in on_chain_heuristics_list}
funding_address_entity, settlement_address_entity = set_mapping(funding_address_entity, settlement_address_entity, on_chain_heuristics)